# Louisville Kentucky Fire Calls

Example project for Code Louisville Data Analysis 1 class.

## Summary

This project uses data from 
[Louisville Metro Open Data](https://data.louisvilleky.gov/) to chart the 
number of service calls by fire districts in the Louisville Metro area.

**Hypothesis:** There are fewer emergency calls at dawn (4am - 8am) than there are 
at dusk (4pm - 8pm).


## Data Discovery

Data Source: [Louisville/Jefferson County Fire Districts calls for 
service.](https://www.arcgis.com/sharing/rest/content/items/a7e8d1790a254d96998ca03e8406afa9/data)

## Data Cleaning

## Data Analysis

## Conclusion